In [53]:
import numpy as np
import math
import random
from graphviz import Digraph

def create_board():
    return np.zeros((3,3) ,dtype=int)

def make_move(board , player, move):
    if board[move[0]][move[1]] == 0:
        board[move[0]][move[1]] = player
        return True
    return False

def check_winner(board, player):
    
    main_diagonal = board.diagonal()
    anti_diagonal = np.fliplr(board).diagonal()
    
    for row in board:
        if np.all(row == player):
            return True
            
    for row in board.T:
         if np.all(row == player):
            return True

    if np.all(main_diagonal == player):
         return True
    
    if np.all(anti_diagonal == player):
         return True
    return False

def switch_player(player):
    if player == 1:
        return -1
    else: return 1

def possible_moves(board):
    moves = []
    indexs = np.where(board == 0)
    for i , j in zip(indexs[0],indexs[1]):
        moves.append((i,j))
    return moves

In [54]:
def create_node(board, parent=None):
    return {
        "state": board,               
        "visited": 0,                 
        "value": 0,                
        "possible_moves": possible_moves(board), 
        "children": [],               
        "parent": parent }


def select_node(node):
    best_ucb = float('-inf')
    best_child = None
    unvisited_children = [child for child in node["children"] if child["visited"] == 0]

    if len(unvisited_children) == len(node["children"]):
        return random.choice(node["children"])

    for child in node["children"]:
        if child["visited"] > 0:
            ucb = child["value"] / child["visited"] + math.sqrt(2) * math.sqrt(math.log(node["visited"]) / child["visited"])
            if ucb > best_ucb:
                best_ucb = ucb
                best_child = child

    return best_child


def expand_node(node, player):
    moves = possible_moves(node["state"])
    node["children"] = []
    
    for move in moves:
        new_board = node["state"].copy()
        make_move(new_board, player, move)
        child_node = create_node(new_board)
        node["children"].append(child_node)


def simulate_game(node, player):
    current_state = node["state"].copy()
    current_player = player
    
    while not check_winner(current_state, current_player) and possible_moves(current_state):
        move = random.choice(possible_moves(current_state))
        make_move(current_state, current_player, move)
        current_player = switch_player(current_player)
    winner = None
    if check_winner(current_state, switch_player(current_player)):
        winner = switch_player(current_player)
    return winner

    
def backpropagate(node, winner, player):
    while node is not None:
        node["visited"] += 1
        if winner == player:
            node["value"] += 1
        node = node["parent"]


In [55]:
def play_game():
    board = create_board()
    player = 1
    current_player = player

    while True:
        print("Current board:")
        print(board)

        if current_player == player:
            row, col = human_move(board)
            make_move(board, current_player, (row, col))
        else:
            root_node = create_node(board)
            for _ in range(1000):
                node = root_node
                current_player_mcts = current_player

                node["visited"] += 1

                while node["children"]:
                    node = select_node(node)
                    current_player_mcts = switch_player(current_player_mcts)

                if not check_winner(node["state"], switch_player(current_player_mcts)):
                    expand_node(node, current_player_mcts)

                if node["children"]:
                    node = random.choice(node["children"])
                    winner = simulate_game(node, current_player_mcts)
                else:
                    winner = None

                backpropagate(node, winner, current_player_mcts)

            best_move = max(root_node["children"], key=lambda x: x["visited"], default=None)
            ai_move = best_move["state"] if best_move else None
            
            if ai_move is not None:
                board = ai_move

        if check_winner(board, current_player):
            print("Player" if current_player == player else "AI", "wins!")            
            break

        if not possible_moves(board):
            
            print("It's a draw!")
            break

        current_player = switch_player(current_player)

def human_move(board):
    while True:
        try:
            row = int(input("Enter row (0-2): "))
            col = int(input("Enter column (0-2): "))
            if board[row][col] == 0:
                return (row, col)
            else:
                print("Cell is already occupied. Try again.")
        except ValueError:
            print("Invalid input. Please enter numbers only.")
        except IndexError:
            print("Row and column must be between 0 and 2.")